In [1]:
import pandas as pd

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from skopt import BayesSearchCV
from skopt.space import Categorical, Integer

# Model training and evaluation
from sklearn.model_selection import train_test_split
#Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

#Metrics
from sklearn.metrics import  accuracy_score

In [2]:
data = pd.read_csv('mbti_twitter.csv', sep="\t", lineterminator='\n')
data

,Unnamed: 0,type,gender,num_tweets,tweets,followers_count,statuses_count,favorites_count,listed_count,ID,E/I,S/N,T/F,J/P
0,0,ENFJ,m,742,back on the way up now at the,8447,16622,21559,121,0.0,E,N,F,J
1,1,ENFJ,m,742,guatnimnwkget your copy now,8447,16622,21559,121,0.0,E,N,F,J
2,2,ENFJ,m,742,Ex8Dz0RxCP thanks for the follow enjoy you...,8447,16622,21559,121,0.0,E,N,F,J
3,3,ENFJ,m,742,rmbxwmyc 9n axel doorman in the back dten...,8447,16622,21559,121,0.0,E,N,F,J
4,4,ENFJ,m,742,1tfzadklqt xxx d all i wanted to say to ...,8447,16622,21559,121,0.0,E,N,F,J
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271987,354,ISTP,f,648,5wvi8zgwm4 en agosto voy a estar en empodera...,660,1125,782,26,1069.0,I,S,T,P
271988,355,ISTP,f,648,idjpjhhjzv me quise hacer la artista a las ...,660,1125,782,26,1069.0,I,S,T,P
271989,356,ISTP,f,648,uh0j41uuqv mejorate D mi socio me di 5 aos...,660,1125,782,26,1069.0,I,S,T,P
271990,357,ISTP,f,648,anspqtq 07i cada vez ms o acuden al para l...,660,1125,782,26,1069.0,I,S,T,P


In [3]:
enc = LabelEncoder()
data['type of encoding'] = enc.fit_transform(data['type'])
target = data['type of encoding'] 

0          0
1          0
2          0
3          0
4          0
          ..
271987    15
271988    15
271989    15
271990    15
271991    15
Name: type of encoding, Length: 271992, dtype: int32

In [5]:
vect = CountVectorizer() 
train =  vect.fit_transform(data["tweets"].astype(str))
train.shape
# (150470, 378898)

(271992, 525229)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.2, stratify=target, random_state=42)
print ((X_train.shape),(y_train.shape),(X_test.shape),(y_test.shape))
#(217593, 525229) (217593,) (54399, 525229) (54399,)

(217593, 525229) (217593,) (54399, 525229) (54399,)


In [7]:
#läuft zu lange: 4311min
param_grid_rf = {
    'n_estimators': Integer(100, 200),                    
    'max_features': Categorical(['sqrt', 'log2', None])                           
}
model=RandomForestClassifier(random_state=42, n_jobs=-1)
RandFor_grid = BayesSearchCV(model, param_grid_rf, scoring='accuracy', verbose=1, cv=2)
RandFor_grid.fit(X_train, y_train)
print(RandFor_grid.best_estimator_)


"\nparam_grid_rf = {\n    'n_estimators': Integer(100, 200),                    \n    'max_features': Categorical(['sqrt', 'log2', None])                           \n}\nmodel=RandomForestClassifier(random_state=42, n_jobs=-1)\nRandFor_grid = BayesSearchCV(model, param_grid_rf, scoring='accuracy', verbose=1, cv=2)\nRandFor_grid.fit(X_train, y_train)\nprint(RandFor_grid.best_estimator_)\n"

In [8]:

param_grid_lr = {                      
    'solver': Categorical(['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']),  
    'max_iter': Integer(100,200)                                   
}
model=LogisticRegression(random_state=42, n_jobs=-1)
logModel_grid = BayesSearchCV(model, param_grid_lr,scoring='accuracy', verbose=1, cv=2)
logModel_grid.fit(X_train, y_train)

#LogisticRegression(class_weight='balanced', max_iter=500, random_state=42, solver='liblinear')
print(logModel_grid.best_estimator_)
#LogisticRegression(max_iter=181, n_jobs=-1, random_state=42, solver='liblinear')


"\nparam_grid_lr = {                      \n    'solver': Categorical(['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']),  \n    'max_iter': Integer(100,200)                                   \n}\nmodel=LogisticRegression(random_state=42, n_jobs=-1)\nlogModel_grid = BayesSearchCV(model, param_grid_lr,scoring='accuracy', verbose=1, cv=2)\nlogModel_grid.fit(X_train, y_train)\n\n#LogisticRegression(class_weight='balanced', max_iter=500, random_state=42, solver='liblinear')\nprint(logModel_grid.best_estimator_)\n#LogisticRegression(max_iter=181, n_jobs=-1, random_state=42, solver='liblinear')\n"

In [10]:
accuracies = {}

In [11]:
#In random forests (see RandomForestClassifier and RandomForestRegressor classes), 
#each tree in the ensemble is built from a sample drawn with replacement (i.e., a bootstrap sample) from the training set.
#Furthermore, when splitting each node during the construction of a tree, the best split is found through an exhaustive search of the features values of either all input features or a random subset of size max_features. 
#The purpose of these two sources of randomness is to decrease the variance of the forest estimator. 

#Random Forest
random_forest = RandomForestClassifier(n_estimators=200, random_state = 42).fit(X_train, y_train)

# make predictions for test data
Y_pred = random_forest.predict(X_test)
predictions = [round(value) for value in Y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
accuracies['Random Forest'] = accuracy* 100.0 
print("Accuracy: %.2f%%" % (accuracy * 100.0))

from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

# 26,56 n_estimators=100, random_state = 1
# 27.66% n_estimators=200, random_state = 42, bootstrap=True


'\n#Random Forest\nrandom_forest = RandomForestClassifier(n_estimators=200, random_state = 42).fit(X_train, y_train)\n\n# make predictions for test data\nY_pred = random_forest.predict(X_test)\npredictions = [round(value) for value in Y_pred]\n\n# evaluate predictions\naccuracy = accuracy_score(y_test, predictions)\naccuracies[\'Random Forest\'] = accuracy* 100.0 \nprint("Accuracy: %.2f%%" % (accuracy * 100.0))\n\nfrom sklearn.metrics import classification_report\nprint(classification_report(y_test, predictions))\n'

In [12]:
logreg = LogisticRegression(max_iter=181, n_jobs=6, random_state=42, solver='liblinear', verbose=1).fit(X_train, y_train)

Y_pred = logreg.predict(X_test)
predictions = [round(value) for value in Y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
accuracies['Logistic Regression'] = accuracy* 100.0
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print(classification_report(y_test, predictions))

# Accuracy: 33.71% class_weight='balanced', max_iter=500, random_state=42, solver='liblinear'
# Accuracy: 34.05% max_iter=181, n_jobs=-1, random_state=42, solver='liblinear', verbose=1
# Accuracy: 32.33% class_weight='balanced', max_iter=181, n_jobs=-1, random_state=42, solver='liblinear', verbose=1

c:\Users\svcsc\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1216: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 6.
  warnings.warn(


[LibLinear]Accuracy: 32.33%
              precision    recall  f1-score   support

           0       0.32      0.43      0.37      8478
           1       0.34      0.32      0.33      5483
           2       0.36      0.34      0.35      6001
           3       0.35      0.34      0.35      2780
           4       0.29      0.36      0.32       955
           5       0.26      0.33      0.29       888
           6       0.31      0.36      0.33      1566
           7       0.19      0.26      0.22       363
           8       0.35      0.31      0.33      6126
           9       0.31      0.24      0.27      4227
          10       0.41      0.31      0.35      8057
          11       0.27      0.27      0.27      3821
          12       0.21      0.26      0.23      1853
          13       0.22      0.24      0.23       507
          14       0.26      0.25      0.25      2588
          15       0.24      0.36      0.29       706

    accuracy                           0.32     5439

c:\Users\svcsc\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [14]:
#This algorithm builds an additive model in a forward stage-wise fashion; 
#it allows for the optimization of arbitrary differentiable loss functions. 
#In each stage n_classes_ regression trees are fit on the negative gradient of the loss function
#GradientBoostingClassifier supports both binary and multi-class classification.

gradientboost = GradientBoostingClassifier(random_state = 42, max_depth=1, verbose= 1).fit(X_train, y_train)

Y_pred = gradientboost.predict(X_test)
predictions = [round(value) for value in Y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
accuracies['Logistic Regression'] = accuracy* 100.0
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print(classification_report(y_test, predictions))


      Iter       Train Loss   Remaining Time 
         1           2.4491         3865.38m
         2           2.4416         3823.61m
         3           2.4370         3793.61m
         4           2.4325         3759.28m
         5           2.4287         3722.98m
         6           2.4255         3674.26m
         7           2.4221         3624.44m
         8           2.4184         3580.28m
         9           2.4160         3539.27m
        10           2.4135         3498.66m
        20           2.3934         3094.19m
        30           2.3789         2725.47m
        40           2.3667         2347.77m
        50           2.3574         1953.50m
        60           2.3483         1561.18m
        70           2.3409         1170.49m
        80           2.3340          779.99m
        90           2.3274          389.86m
       100           2.3209            0.00s
Accuracy: 22.72%
              precision    recall  f1-score   support

           0       0.18    

: 